# Cross-Method Validation Notebook (REHAB24-6)

This notebook reproduces the **exact same REHAB24-6 angle representation** used in `MQM_REHAB24_6_final.ipynb`, but swaps the classifiers to:

- **XGBoost** (replaces Linear Regression baseline)
- **LSTM** (replaces Dual-Attention TCN)

Scope is intentionally minimal: **no attention, no explainability, no stats tests, no robustness checks**.

> Key constraint: both models consume the same 10-angle time series: `X.shape = (N, 100, 10)`.


In [1]:
# --- Imports & Reproducibility ---
import os, re, glob, math, random
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print('OK: imports')


OK: imports


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

REHAB_ROOT = Path("/content/drive/MyDrive/REHAB24-6")   # <-- change
SEG_CSV    = REHAB_ROOT / "Segmentation.csv"
JOINTS_TXT = REHAB_ROOT / "joints_names.txt"

# Choose ONE:
JOINTS_DIR = REHAB_ROOT / "2d_joints"   # or REHAB_ROOT / "3d_joints"

TARGET_LEN = 100
USE_2D     = True

seg = pd.read_csv(SEG_CSV, sep=";")

with open(JOINTS_TXT, "r", encoding="utf-8") as f:
    joint_names = [line.strip() for line in f if line.strip()]
name_to_idx = {n:i for i,n in enumerate(joint_names)}

print("seg:", seg.shape, " | joints:", len(joint_names))
print("JOINTS_DIR exists:", JOINTS_DIR.exists())

seg: (1072, 13)  | joints: 26
JOINTS_DIR exists: True


In [4]:
# --- Load joint names (26) and segmentation table ---
with open(JOINTS_TXT, 'r', encoding='utf-8') as f:
    joint_names = [line.strip() for line in f if line.strip()]

name_to_idx = {n:i for i,n in enumerate(joint_names)}
print('Num joints:', len(joint_names))
print('First 10 joints:', joint_names[:10])

# Segmentation is semicolon-separated in REHAB24
seg = pd.read_csv(SEG_CSV, sep=';')
print('Segmentation rows:', len(seg))
seg.head()


Num joints: 26
First 10 joints: ['0: Hips', '1: Spine', '2: Spine1', '3: Neck', '4: Head', '5: Head_end', '6: LeftShoulder', '7: LeftArm', '8: LeftForeArm', '9: LeftHand']
Segmentation rows: 1072


,video_id,repetition_number,exercise_id,person_id,first_frame,last_frame,cam17_orientation,mocap_erroneous,exercise_subtype,lights_on,extra_person_in_cam17,extra_person_in_cam18,correctness
0,PM_000,1,1,1,180,377,front,0,right arm,0,3,0,1
1,PM_000,2,1,1,378,620,front,0,right arm,0,3,0,1
2,PM_000,3,1,1,621,865,front,0,right arm,0,3,0,1
3,PM_000,4,1,1,866,1085,front,0,right arm,0,3,3,1
4,PM_000,5,1,1,1086,1265,front,0,right arm,0,3,3,1


In [5]:
def find_joint_file(video_id: str, joints_dir: Path) -> Path | None:
    """
    Finds a joints file for a given video_id.
    Tries common naming patterns and falls back to recursive search.
    """
    vid = str(video_id)

    # common direct candidates
    candidates = [
        joints_dir / f"{vid}.npy",
        joints_dir / f"{vid}.npz",
        joints_dir / f"{vid}_joints.npy",
        joints_dir / f"{vid}_joints.npz",
    ]
    for p in candidates:
        if p.exists():
            return p

    # fallback: recursive contains match
    for ext in ("*.npy", "*.npz"):
        for p in joints_dir.rglob(ext):
            if vid in p.stem:
                return p
    return None


def load_joints_array(path: Path) -> np.ndarray:
    """
    Returns joints as ndarray with shape (T, J, D) or (T, J, 3/2).
    """
    if path.suffix == ".npy":
        arr = np.load(path, allow_pickle=True)
        return arr
    if path.suffix == ".npz":
        z = np.load(path, allow_pickle=True)
        # common keys: 'arr_0', 'joints', 'keypoints'
        for k in ("joints", "keypoints", "arr_0"):
            if k in z:
                return z[k]
        # last resort: first key
        return z[list(z.keys())[0]]

    raise ValueError(f"Unsupported joints file type: {path}")

In [6]:
import numpy as np
from collections import Counter
from tqdm import tqdm

import numpy as np
from collections import Counter
from tqdm import tqdm

def build_rehab24_angles_dataset_from_files(
    seg_df: pd.DataFrame,
    name_to_idx: dict,
    joints_dir: Path,
    target_len: int = 100,
    use_2d: bool = True,
    drop_mocap_erroneous: bool = True,
):
    X_list, y_list, g_list, meta_rows = [], [], [], []
    reasons = Counter()

    for row in tqdm(seg_df.itertuples(index=False), total=len(seg_df), desc="Building angles"):
        video_id = str(row.video_id)
        person_id = str(row.person_id)
        y = int(row.correctness)

        # optional filtering
        if drop_mocap_erroneous and hasattr(row, "mocap_erroneous"):
            try:
                if int(row.mocap_erroneous) == 1:
                    reasons["drop_mocap_erroneous"] += 1
                    continue
            except Exception:
                pass

        p = find_joint_file(video_id, joints_dir)
        if p is None:
            reasons["missing_joint_file"] += 1
            continue

        joints = load_joints_array(p)

        # Expect (T,J,D)
        if joints.ndim != 3:
            reasons["bad_joints_shape"] += 1
            continue

        T = joints.shape[0]
        ff = int(row.first_frame)
        lf = int(row.last_frame)

        # clamp and slice (treat last_frame as inclusive)
        ff = max(0, min(ff, T-1))
        lf = max(0, min(lf, T-1))
        if lf < ff:
            reasons["bad_frame_range"] += 1
            continue

        joints_seg = joints[ff:lf+1]

        # --- your existing functions from the notebook ---
        coco = remap_to_coco_like(joints_seg, name_to_idx=name_to_idx)
        if coco is None or not np.isfinite(coco).any():
            reasons["coco_bad"] += 1
            continue

        angles = compute_angles_sequence(coco, target_len=target_len, use_2d=use_2d)  # (target_len, 10)
        if angles is None or angles.shape[0] != target_len:
            reasons["angles_bad"] += 1
            continue

        if not np.isfinite(angles).any():
            reasons["angles_all_nan"] += 1
            continue

        # allow a bit of NaNs, then fill
        if np.isnan(angles).mean() > 0.30:
            reasons["angles_too_many_nans"] += 1
            continue
        angles = np.nan_to_num(angles, nan=0.0, posinf=0.0, neginf=0.0)

        X_list.append(angles.astype(np.float32))
        y_list.append(y)
        g_list.append(person_id)
        meta_rows.append({
            "video_id": video_id,
            "person_id": person_id,
            "exercise_id": int(row.exercise_id),
            "rep": int(row.repetition_number),
            "first_frame": ff,
            "last_frame": lf,
            "label": y,
            "joint_file": str(p),
        })

    print("\nTop skip reasons:", reasons.most_common(15))

    if len(X_list) == 0:
        raise ValueError("No valid samples produced. Reasons: " + str(reasons.most_common(10)))

    X = np.stack(X_list, axis=0)                # (N, 100, 10)
    y = np.array(y_list, dtype=np.int64)        # (N,)
    groups = np.array(g_list)                   # (N,)
    meta = pd.DataFrame(meta_rows)
    return X, y, groups, meta

In [17]:
import numpy as np

COCO17 = [
    "nose","left_eye","right_eye","left_ear","right_ear",
    "left_shoulder","right_shoulder","left_elbow","right_elbow",
    "left_wrist","right_wrist","left_hip","right_hip",
    "left_knee","right_knee","left_ankle","right_ankle"
]

# Map COCO joint -> your dataset joint name (26-joint)
COCO_TO_REHAB = {
    "nose": "Head",  # best proxy
    "left_eye": "Head", "right_eye": "Head",
    "left_ear": "Head", "right_ear": "Head",

    "left_shoulder": "LeftShoulder",
    "right_shoulder": "RightShoulder",

    "left_elbow": "LeftArm",
    "right_elbow": "RightArm",

    "left_wrist": "LeftHand",
    "right_wrist": "RightHand",

    "left_hip": "LeftUpLeg",
    "right_hip": "RightUpLeg",

    "left_knee": "LeftLeg",
    "right_knee": "RightLeg",

    "left_ankle": "LeftFoot",
    "right_ankle": "RightFoot",
}

def remap_to_coco_like(joints: np.ndarray, name_to_idx: dict) -> np.ndarray:
    """
    joints: (T, 26, D) where D is 2 or 3
    returns: (T, 17, D) in COCO order above.
    Missing joints become NaN.
    """
    T, J, D = joints.shape
    out = np.full((T, 17, D), np.nan, dtype=np.float32)

    for coco_i, coco_name in enumerate(COCO17):
        src_name = COCO_TO_REHAB.get(coco_name, None)
        if src_name is None:
            continue
        if src_name not in name_to_idx:
            # leave NaN
            continue
        out[:, coco_i, :] = joints[:, name_to_idx[src_name], :]

    return out

In [18]:
def clean_joint_name(s: str) -> str:
    s = s.strip()
    if ":" in s:
        s = s.split(":", 1)[1].strip()
    return s

with open(JOINTS_TXT, "r", encoding="utf-8") as f:
    joint_names_raw = [line.strip() for line in f if line.strip()]

joint_names = [clean_joint_name(x) for x in joint_names_raw]
name_to_idx = {n:i for i,n in enumerate(joint_names)}

print("First 10 cleaned joints:", joint_names[:10])
print("Has LeftShoulder?", "LeftShoulder" in name_to_idx)

First 10 cleaned joints: ['Hips', 'Spine', 'Spine1', 'Neck', 'Head', 'Head_end', 'LeftShoulder', 'LeftArm', 'LeftForeArm', 'LeftHand']
Has LeftShoulder? True


In [8]:
def _vec(a, b):
    return b - a

def _angle(u, v, eps=1e-8):
    # angle between vectors u and v, per frame
    num = np.sum(u*v, axis=-1)
    den = (np.linalg.norm(u, axis=-1) * np.linalg.norm(v, axis=-1)) + eps
    cos = np.clip(num/den, -1.0, 1.0)
    return np.arccos(cos)

def _joint_angle(A, B, C):
    # angle at B: BA vs BC
    return _angle(_vec(B, A), _vec(B, C))

def resample_to_len(seq: np.ndarray, target_len: int) -> np.ndarray:
    """
    seq: (T, A) -> (target_len, A) via linear interpolation on time axis.
    """
    T, A = seq.shape
    if T == target_len:
        return seq
    if T < 2:
        return None

    x_old = np.linspace(0, 1, T)
    x_new = np.linspace(0, 1, target_len)
    out = np.zeros((target_len, A), dtype=np.float32)

    for a in range(A):
        y = seq[:, a]
        # interpolate NaNs safely: fill gaps by nearest then interpolate
        if np.isnan(y).all():
            out[:, a] = np.nan
            continue
        # forward/back fill
        yy = y.copy()
        idx = np.where(~np.isnan(yy))[0]
        yy[:idx[0]] = yy[idx[0]]
        yy[idx[-1]+1:] = yy[idx[-1]]
        # linear interp for internal NaNs
        nidx = np.where(np.isnan(yy))[0]
        if len(nidx) > 0:
            yy[nidx] = np.interp(nidx, idx, yy[idx])
        out[:, a] = np.interp(x_new, x_old, yy).astype(np.float32)

    return out

def compute_angles_sequence(coco17: np.ndarray, target_len=100, use_2d=True) -> np.ndarray:
    """
    coco17: (T,17,D). Returns (target_len,10) angles.
    """
    T, J, D = coco17.shape
    if use_2d and D > 2:
        P = coco17[:, :, :2].astype(np.float32)
    else:
        P = coco17.astype(np.float32)

    # COCO indices
    LS, RS = 5, 6
    LE, RE = 7, 8
    LW, RW = 9, 10
    LH, RH = 11, 12
    LK, RK = 13, 14
    LA, RA = 15, 16

    # elbow flexion: shoulder-elbow-wrist
    a_L_elbow = _joint_angle(P[:, LS], P[:, LE], P[:, LW])
    a_R_elbow = _joint_angle(P[:, RS], P[:, RE], P[:, RW])

    # shoulder angle: elbow-shoulder-hip (arm relative to trunk)
    a_L_shoulder = _joint_angle(P[:, LE], P[:, LS], P[:, LH])
    a_R_shoulder = _joint_angle(P[:, RE], P[:, RS], P[:, RH])

    # hip angle: shoulder-hip-knee (thigh vs trunk)
    a_L_hip = _joint_angle(P[:, LS], P[:, LH], P[:, LK])
    a_R_hip = _joint_angle(P[:, RS], P[:, RH], P[:, RK])

    # knee flexion: hip-knee-ankle
    a_L_knee = _joint_angle(P[:, LH], P[:, LK], P[:, LA])
    a_R_knee = _joint_angle(P[:, RH], P[:, RK], P[:, RA])

    # torso deviations: use hip-mid and shoulder-mid vector vs vertical axis
    hip_mid = 0.5 * (P[:, LH] + P[:, RH])
    sh_mid  = 0.5 * (P[:, LS] + P[:, RS])
    torso_v = sh_mid - hip_mid  # (T,2)

    # vertical reference (image y axis down); use (0,-1) as "up"
    v_ref = np.zeros_like(torso_v)
    v_ref[:, 1] = -1.0

    torso_angle = _angle(torso_v, v_ref)  # (T,)
    # split into two features like your previous "torso_L_dev / torso_R_dev"
    # using shoulder asymmetry direction
    sh_vec = P[:, RS] - P[:, LS]
    dev_sign = np.sign(sh_vec[:, 0] + 1e-6)  # left/right leaning proxy
    torso_L_dev = torso_angle * (dev_sign < 0)
    torso_R_dev = torso_angle * (dev_sign >= 0)

    angles = np.stack([
        a_L_elbow, a_R_elbow,
        a_L_shoulder, a_R_shoulder,
        a_L_hip, a_R_hip,
        a_L_knee, a_R_knee,
        torso_L_dev, torso_R_dev
    ], axis=1).astype(np.float32)  # (T,10)

    return resample_to_len(angles, target_len)

In [15]:
seg

,video_id,repetition_number,exercise_id,person_id,first_frame,last_frame,cam17_orientation,mocap_erroneous,exercise_subtype,lights_on,extra_person_in_cam17,extra_person_in_cam18,correctness
0,PM_000,1,1,1,180,377,front,0,right arm,0,3,0,1
1,PM_000,2,1,1,378,620,front,0,right arm,0,3,0,1
2,PM_000,3,1,1,621,865,front,0,right arm,0,3,0,1
3,PM_000,4,1,1,866,1085,front,0,right arm,0,3,3,1
4,PM_000,5,1,1,1086,1265,front,0,right arm,0,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,PM_126,16,6,7,2630,2730,half-profile,0,NaN,1,0,0,0
1068,PM_126,17,6,7,2750,2860,half-profile,0,NaN,1,1,0,0
1069,PM_126,18,6,7,2880,2980,half-profile,0,NaN,1,0,0,0
1070,PM_126,19,6,7,3000,3110,half-profile,0,NaN,1,0,0,0


In [19]:
X, y, groups, meta = build_rehab24_angles_dataset_from_files(
    seg_df=seg,
    name_to_idx=name_to_idx,
    joints_dir=JOINTS_DIR,
    target_len=TARGET_LEN,
    use_2d=USE_2D,
    drop_mocap_erroneous=True,
)

print("X shape:", X.shape)         # (N,100,10)
print("y:", y.shape, "labels:", np.unique(y, return_counts=True))
print("groups:", groups.shape, "unique persons:", len(np.unique(groups)))
display(meta.head())

Building angles: 100%|██████████| 1072/1072 [00:10<00:00, 98.67it/s] 



Top skip reasons: [('drop_mocap_erroneous', 15)]
X shape: (1057, 100, 10)
y: (1057,) labels: (array([0, 1]), array([499, 558]))
groups: (1057,) unique persons: 10


,video_id,person_id,exercise_id,rep,first_frame,last_frame,label,joint_file
0,PM_000,1,1,1,180,377,1,/content/drive/MyDrive/REHAB24-6/2d_joints/Ex1...
1,PM_000,1,1,2,378,620,1,/content/drive/MyDrive/REHAB24-6/2d_joints/Ex1...
2,PM_000,1,1,3,621,865,1,/content/drive/MyDrive/REHAB24-6/2d_joints/Ex1...
3,PM_000,1,1,4,866,1085,1,/content/drive/MyDrive/REHAB24-6/2d_joints/Ex1...
4,PM_000,1,1,5,1086,1265,1,/content/drive/MyDrive/REHAB24-6/2d_joints/Ex1...


In [20]:
# --- REHAB24 joint-index -> COCO17 mapping (kept for parity with main notebook) ---
# Note: models here consume ANGLES, not COCO joints. We keep this mapping as a reference/checkpoint.

REHAB24_TO_COCO = {
    'Head': 0,                 # nose proxy
    'LeftShoulder': 5,
    'RightShoulder': 6,
    'LeftForeArm': 7,          # elbow proxy
    'RightForeArm': 8,
    'LeftHand': 9,             # wrist proxy
    'RightHand': 10,
    'LeftUpLeg': 11,           # hip proxy
    'RightUpLeg': 12,
    'LeftLeg': 13,             # knee proxy
    'RightLeg': 14,
    'LeftFoot': 15,            # ankle proxy
    'RightFoot': 16,
}

missing = [k for k in REHAB24_TO_COCO.keys() if k not in name_to_idx]
print('Missing REHAB24 joint names (should be empty):', missing)


Missing REHAB24 joint names (should be empty): []


In [21]:
# --- Subject-disjoint CV splits (Generalized Regime) ---
N_SPLITS = 5

gkf = GroupKFold(n_splits=N_SPLITS)
folds = list(gkf.split(X, y, groups=groups))
print('Num folds:', len(folds))

# Quick sanity: no subject leakage
for k,(tr,te) in enumerate(folds, start=1):
    inter = set(groups[tr]).intersection(set(groups[te]))
    print(f'Fold {k}: train N={len(tr)} test N={len(te)} leakage subjects={len(inter)}')


Num folds: 5
Fold 1: train N=922 test N=135 leakage subjects=0
Fold 2: train N=828 test N=229 leakage subjects=0
Fold 3: train N=830 test N=227 leakage subjects=0
Fold 4: train N=824 test N=233 leakage subjects=0
Fold 5: train N=824 test N=233 leakage subjects=0


In [14]:
# --- Model 1: XGBoost on flattened angles ---
# We flatten (T,10) -> (T*10,) to mirror the classic feature-vector baseline.

# Install if needed:
# !pip -q install xgboost

from xgboost import XGBClassifier

X_flat = X.reshape(X.shape[0], -1)

xgb_params = dict(
    n_estimators=400,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    min_child_weight=1,
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=SEED,
    n_jobs=-1,
)

xgb_fold_ba = []
xgb_reports = {}

for k,(tr,te) in enumerate(folds, start=1):
    clf = XGBClassifier(**xgb_params)
    clf.fit(X_flat[tr], y[tr])

    pred = (clf.predict_proba(X_flat[te])[:,1] >= 0.5).astype(int)
    ba = balanced_accuracy_score(y[te], pred)
    xgb_fold_ba.append(ba)

    xgb_reports[k] = classification_report(y[te], pred, digits=3)
    print(f'Fold {k} BA: {ba:.3f}')

print('\nXGBoost BA mean±std:', float(np.mean(xgb_fold_ba)), float(np.std(xgb_fold_ba)))


Fold 1 BA: 0.535
Fold 2 BA: 0.536
Fold 3 BA: 0.527
Fold 4 BA: 0.548
Fold 5 BA: 0.502

XGBoost BA mean±std: 0.5299195629807878 0.015492508408075322


In [15]:
# --- Model 2: LSTM on (T,10) sequence ---
# PyTorch implementation: simple BiLSTM + mean pooling + linear head.


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('DEVICE:', DEVICE)

class AnglesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class LSTMClassifier(nn.Module):
    def __init__(self, input_size=10, hidden_size=64, num_layers=2, bidirectional=True, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        out_dim = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.LayerNorm(out_dim),
            nn.Dropout(dropout),
            nn.Linear(out_dim, 2)
        )

    def forward(self, x):
        # x: (B,T,F)
        h, _ = self.lstm(x)
        # mean pool over time
        h = h.mean(dim=1)
        return self.head(h)


def train_one_fold_lstm(X_tr, y_tr, X_va, y_va, seed=SEED, epochs=40, batch_size=64, lr=1e-3, patience=8):
    torch.manual_seed(seed)
    model = LSTMClassifier().to(DEVICE)

    train_loader = DataLoader(AnglesDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(AnglesDataset(X_va, y_va), batch_size=batch_size, shuffle=False)

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    crit = nn.CrossEntropyLoss()

    best_ba = -1.0
    best_state = None
    bad = 0

    for ep in range(1, epochs+1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)
            logits = model(xb)
            loss = crit(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()

        # val
        model.eval()
        all_p, all_t = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(DEVICE)
                logits = model(xb)
                pred = torch.argmax(logits, dim=1).cpu().numpy()
                all_p.append(pred)
                all_t.append(yb.numpy())
        all_p = np.concatenate(all_p)
        all_t = np.concatenate(all_t)
        ba = balanced_accuracy_score(all_t, all_p)

        if ba > best_ba + 1e-4:
            best_ba = ba
            best_state = {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1

        if bad >= patience:
            break

    # load best
    if best_state is not None:
        model.load_state_dict(best_state)
    return model


lstm_fold_ba = []
lstm_reports = {}

for k,(tr,te) in enumerate(folds, start=1):
    model = train_one_fold_lstm(X[tr], y[tr], X[te], y[te])

    model.eval()
    with torch.no_grad():
        xb = torch.tensor(X[te], dtype=torch.float32).to(DEVICE)
        logits = model(xb).cpu().numpy()
        pred = np.argmax(logits, axis=1)

    ba = balanced_accuracy_score(y[te], pred)
    lstm_fold_ba.append(ba)
    lstm_reports[k] = classification_report(y[te], pred, digits=3)
    print(f'Fold {k} BA: {ba:.3f}')

print('\nLSTM BA mean±std:', float(np.mean(lstm_fold_ba)), float(np.std(lstm_fold_ba)))


DEVICE: cuda
Fold 1 BA: 0.603
Fold 2 BA: 0.572
Fold 3 BA: 0.552
Fold 4 BA: 0.607
Fold 5 BA: 0.593

LSTM BA mean±std: 0.5852907360622459 0.020424608289205677


In [16]:
# --- Summary table ---
import pandas as pd

summary = pd.DataFrame({
    'fold': list(range(1, len(folds)+1)),
    'XGBoost_BA': xgb_fold_ba,
    'LSTM_BA': lstm_fold_ba,
})
summary.loc['mean'] = ['mean', np.mean(xgb_fold_ba), np.mean(lstm_fold_ba)]
summary.loc['std']  = ['std',  np.std(xgb_fold_ba),  np.std(lstm_fold_ba)]
summary


,fold,XGBoost_BA,LSTM_BA
0,1,0.535490,0.602665
1,2,0.536415,0.572324
2,3,0.527327,0.552082
3,4,0.548369,0.606700
4,5,0.501996,0.592683
mean,mean,0.529920,0.585291
std,std,0.015493,0.020425


In [17]:
import numpy as np
import pandas as pd

# seg: your segmentation dataframe
# groups: array aligned with X (person_id per sample)
# meta: dataframe returned by the builder (must include person_id, exercise_id, label)

print("---- BASIC DATASET COUNTS ----")
print("N samples:", len(meta))
print("Unique persons:", meta["person_id"].nunique())
print("Unique exercises:", meta["exercise_id"].nunique())
print("Label counts:\n", meta["label"].value_counts().sort_index())

print("\n---- SAMPLES PER PERSON ----")
per_person = meta.groupby("person_id").size().sort_values(ascending=False)
print(per_person.describe())
print("Top 10 persons by samples:\n", per_person.head(10))

print("\n---- SAMPLES PER (PERSON, EXERCISE) PAIR ----")
pair_counts = meta.groupby(["person_id", "exercise_id"]).size().sort_values(ascending=False)
print(pair_counts.describe())
print("Top 20 pairs by samples:\n", pair_counts.head(20))

print("\n---- CLASS BALANCE PER (PERSON, EXERCISE) PAIR ----")
pair_label_counts = (
    meta.groupby(["person_id", "exercise_id", "label"])
        .size()
        .unstack(fill_value=0)
)

# ensure both columns 0 and 1 exist
for c in [0, 1]:
    if c not in pair_label_counts.columns:
        pair_label_counts[c] = 0

pair_label_counts["n_total"] = pair_label_counts[0] + pair_label_counts[1]
pair_label_counts["minority"] = pair_label_counts[[0, 1]].min(axis=1)
pair_label_counts["majority"] = pair_label_counts[[0, 1]].max(axis=1)
pair_label_counts["minority_frac"] = pair_label_counts["minority"] / pair_label_counts["n_total"].replace(0, np.nan)

print(pair_label_counts[["n_total", 0, 1, "minority", "minority_frac"]].describe())

print("\nPairs with only one class (bad for individualized):",
      int((pair_label_counts["minority"] == 0).sum()))

print("Pairs with minority < 2 (usually too small for stratified split):",
      int((pair_label_counts["minority"] < 2).sum()))

print("Pairs usable if we require minority >= 2:",
      int((pair_label_counts["minority"] >= 2).sum()))

print("\nTop 20 most balanced (highest minority_frac, with enough samples):")
balanced = pair_label_counts[pair_label_counts["n_total"] >= 10].sort_values("minority_frac", ascending=False)
print(balanced.head(20)[["n_total", 0, 1, "minority_frac"]])

print("\nTop 20 most imbalanced (lowest minority_frac, with enough samples):")
imbal = pair_label_counts[pair_label_counts["n_total"] >= 10].sort_values("minority_frac", ascending=True)
print(imbal.head(20)[["n_total", 0, 1, "minority_frac"]])

---- BASIC DATASET COUNTS ----
N samples: 1057
Unique persons: 10
Unique exercises: 6
Label counts:
 label
0    499
1    558
Name: count, dtype: int64

---- SAMPLES PER PERSON ----
count     10.000000
mean     105.700000
std       35.074682
min        8.000000
25%      108.250000
50%      116.500000
75%      121.750000
max      127.000000
dtype: float64
Top 10 persons by samples:
 person_id
9     127
1     123
3     123
8     118
5     117
2     116
4     115
6     106
7     104
10      8
dtype: int64

---- SAMPLES PER (PERSON, EXERCISE) PAIR ----
count    53.000000
mean     19.943396
std       5.168005
min       8.000000
25%      20.000000
50%      21.000000
75%      23.000000
max      30.000000
dtype: float64
Top 20 pairs by samples:
 person_id  exercise_id
1          4              30
           6              27
3          4              27
1          2              26
2          2              25
9          2              25
8          5              25
5          5              2

# LSTM and XGB

In [45]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold, StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# X: (N,100,10)
# y: (N,)
# groups: (N,) person_id aligned with X
# meta: dataframe aligned with X (same row order as X_list appends)
# meta columns: ['person_id','exercise_id','label', ...]
assert len(X) == len(y) == len(groups) == len(meta)

In [40]:
from xgboost import XGBClassifier

def fit_xgb(X_train, y_train, seed=42):
    # flatten (N,100,10) -> (N,1000)
    Xtr = X_train.reshape(len(X_train), -1)

    # handle imbalance with scale_pos_weight (optional but sensible)
    n_pos = (y_train == 1).sum()
    n_neg = (y_train == 0).sum()
    spw = (n_neg / max(n_pos, 1))

    model = XGBClassifier(
        n_estimators=600,
        max_depth=4,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        min_child_weight=1.0,
        gamma=0.0,
        objective="binary:logistic",
        eval_metric="logloss",
        tree_method="hist",
        random_state=seed,
        n_jobs=-1,
        scale_pos_weight=spw,
    )
    model.fit(Xtr, y_train)
    return model

def predict_xgb(model, X_test):
    Xte = X_test.reshape(len(X_test), -1)
    proba = model.predict_proba(Xte)[:, 1]
    pred = (proba >= 0.5).astype(int)
    return pred

In [38]:
import tensorflow as tf
from tensorflow import keras

def build_lstm(input_shape=(100,10), seed=42):
    tf.keras.utils.set_random_seed(int(seed))
    inp = keras.Input(shape=input_shape)
    x = keras.layers.Masking(mask_value=0.0)(inp)
    x = keras.layers.LSTM(32, return_sequences=True)(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.LSTM(16)(x)
    x = keras.layers.Dropout(0.2)(x)
    out = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=[]
    )
    return model

def fit_lstm(X_train, y_train, X_val, y_val, seed=42, max_epochs=80, batch_size=32):
    model = build_lstm(input_shape=X_train.shape[1:], seed=seed)

    # class weights
    classes = np.array([0,1])
    cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
    class_weight = {0: float(cw[0]), 1: float(cw[1])}

    cb = [
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.5, min_lr=1e-5),
    ]

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        batch_size=batch_size,
        verbose=0,
        class_weight=class_weight,
        callbacks=cb,
    )
    return model

def predict_lstm(model, X_test):
    proba = model.predict(X_test, verbose=0).ravel()
    pred = (proba >= 0.5).astype(int)
    return pred

In [43]:
def run_generalized(X, y, groups, seed=42, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)

    rows = []
    fold = 0
    for tr_idx, te_idx in gkf.split(X, y, groups):
        fold += 1
        Xtr, ytr = X[tr_idx], y[tr_idx]
        Xte, yte = X[te_idx], y[te_idx]

        # make a small validation split from train for LSTM early stopping
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed+fold)
        tr2_idx, va_idx = next(sss.split(Xtr, ytr))
        Xtr2, ytr2 = Xtr[tr2_idx], ytr[tr2_idx]
        Xva, yva = Xtr[va_idx], ytr[va_idx]

        # XGB
        xgb = fit_xgb(Xtr, ytr, seed=seed+fold)
        yhat_xgb = predict_xgb(xgb, Xte)
        ba_xgb = balanced_accuracy_score(yte, yhat_xgb)

        # LSTM
        lstm = fit_lstm(Xtr2, ytr2, Xva, yva, seed=seed+fold)
        yhat_lstm = predict_lstm(lstm, Xte)
        ba_lstm = balanced_accuracy_score(yte, yhat_lstm)

        rows.append({
            "fold": fold,
            "n_test": len(te_idx),
            "test_subjects": len(np.unique(groups[te_idx])),
            "BA_XGB": ba_xgb,
            "BA_LSTM": ba_lstm,
            "Delta_XGB_minus_LSTM": ba_xgb - ba_lstm
        })

    df = pd.DataFrame(rows)
    print(df)
    print("\nMean±std:")
    print("XGB :", df["BA_XGB"].mean(), df["BA_XGB"].std())
    print("LSTM:", df["BA_LSTM"].mean(), df["BA_LSTM"].std())
    print("Δ   :", df["Delta_XGB_minus_LSTM"].mean(), df["Delta_XGB_minus_LSTM"].std())
    return df

In [51]:
gen_df = run_generalized(X, y, groups, seed=42, n_splits=5)

   fold  n_test  test_subjects    BA_XGB   BA_LSTM  Delta_XGB_minus_LSTM
0     1     135              2  0.554971  0.500112              0.054859
1     2     229              2  0.517253  0.616607             -0.099353
2     3     227              2  0.508459  0.476571              0.031888
3     4     233              2  0.522248  0.483840              0.038408
4     5     233              2  0.549593  0.442905              0.106689

Mean±std:
XGB : 0.5305049742990573 0.02057150369943565
LSTM: 0.5040069225326995 0.06630772126707156
Δ   : 0.026498051766357722 0.07623057728649361


In [52]:
mask = meta["person_id"].astype(str) != "10"
gen_df_no10 = run_generalized(X[mask], y[mask], groups[mask], seed=42, n_splits=5)

   fold  n_test  test_subjects    BA_XGB   BA_LSTM  Delta_XGB_minus_LSTM
0     1     127              1  0.554798  0.472727              0.082071
1     2     229              2  0.542842  0.588877             -0.046035
2     3     227              2  0.520856  0.562802             -0.041946
3     4     233              2  0.518447  0.491145              0.027302
4     5     233              2  0.551035  0.502772              0.048263

Mean±std:
XGB : 0.5375955633508841 0.01696257186595277
LSTM: 0.5236646005091437 0.04968151334400789
Δ   : 0.013930962841740524 0.05638859527894348


In [10]:
import tensorflow as tf
from tensorflow import keras

def build_lstm_fast(input_shape=(100,10), seed=42):
    tf.keras.utils.set_random_seed(int(seed))

    inp = keras.Input(shape=input_shape)
    x = keras.layers.Masking(mask_value=0.0)(inp)
    x = keras.layers.LSTM(16)(x)              # smaller
    x = keras.layers.Dropout(0.2)(x)
    out = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out)
    model.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss="binary_crossentropy")
    return model

def fit_lstm_fast(X_train, y_train, seed=42, max_epochs=30, batch_size=64):
    tf.keras.backend.clear_session()  # IMPORTANT: prevents slow-down over many fits
    model = build_lstm_fast(input_shape=X_train.shape[1:], seed=seed)

    # validation_split avoids extra split logic overhead
    cb = [
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.5, min_lr=1e-5),
    ]
    model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=max_epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=cb,
        shuffle=True
    )
    return model

In [11]:
def predict_lstm(model, X_test):
    proba = model.predict(X_test, verbose=0).ravel()
    return (proba >= 0.5).astype(int)

In [12]:
from xgboost import XGBClassifier

def fit_xgb_fast(X_train, y_train, seed=42):
    Xtr = X_train.reshape(len(X_train), -1)
    n_pos = (y_train == 1).sum()
    n_neg = (y_train == 0).sum()
    spw = (n_neg / max(n_pos, 1))

    model = XGBClassifier(
        n_estimators=250,      # lower
        max_depth=3,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        tree_method="hist",
        random_state=seed,
        n_jobs=-1,
        scale_pos_weight=spw,
        eval_metric="logloss",
    )
    model.fit(Xtr, y_train)
    return model

def predict_xgb(model, X_test):
    Xte = X_test.reshape(len(X_test), -1)
    proba = model.predict_proba(Xte)[:, 1]
    return (proba >= 0.5).astype(int)

In [13]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score

def run_individualized_fast(X, y, meta, seed=42, n_repeats=5, min_minority=2):
    meta2 = meta.copy()
    meta2["person_id"] = meta2["person_id"].astype(str)
    meta2["exercise_id"] = meta2["exercise_id"].astype(int)

    groups = meta2.groupby(["person_id", "exercise_id"]).indices
    results = []

    for (pid, exid), idxs in tqdm(groups.items(), total=len(groups), desc="Pairs"):
        idxs = np.array(idxs)
        yy = y[idxs]
        n = len(idxs)

        n0 = int((yy==0).sum()); n1 = int((yy==1).sum())
        minority = min(n0, n1)
        if minority < min_minority:
            continue

        # test size must be >= 2 and allow both classes
        test_size = max(2, int(0.3 * n))
        if test_size >= n:
            continue

        # if too small to stratify robustly, skip
        if test_size < 2:
            continue

        sss = StratifiedShuffleSplit(
            n_splits=n_repeats,
            test_size=test_size,
            random_state=seed + (hash(pid) % 10000) + exid
        )

        bas_xgb, bas_lstm = [], []

        for r, (tr_local, te_local) in enumerate(sss.split(np.zeros(n), yy)):
            tr_idx = idxs[tr_local]
            te_idx = idxs[te_local]

            Xtr, ytr = X[tr_idx], y[tr_idx]
            Xte, yte = X[te_idx], y[te_idx]

            # XGB
            xgb = fit_xgb_fast(Xtr, ytr, seed=seed + r + exid)
            yhat_xgb = predict_xgb(xgb, Xte)
            bas_xgb.append(balanced_accuracy_score(yte, yhat_xgb))

            # LSTM (fast)
            lstm = fit_lstm_fast(Xtr, ytr, seed=seed + r + exid)
            yhat_lstm = predict_lstm(lstm, Xte)
            bas_lstm.append(balanced_accuracy_score(yte, yhat_lstm))

        results.append({
            "person_id": pid,
            "exercise_id": exid,
            "n_total": n,
            "n0": n0,
            "n1": n1,
            "BA_XGB_mean": float(np.mean(bas_xgb)),
            "BA_LSTM_mean": float(np.mean(bas_lstm)),
            "Delta_mean": float(np.mean(bas_xgb) - np.mean(bas_lstm)),
        })

    df = pd.DataFrame(results).sort_values("Delta_mean", ascending=False).reset_index(drop=True)
    print("Usable pairs:", len(df))
    print("Mean BA over pairs: XGB =", df["BA_XGB_mean"].mean(), " | LSTM =", df["BA_LSTM_mean"].mean())
    print("Median Δ (XGB-LSTM):", df["Delta_mean"].median())
    print("% pairs XGB>LSTM:", (df["Delta_mean"] > 0).mean())
    return df

In [22]:
ind_df_fast = run_individualized_fast(X, y, meta, seed=42, n_repeats=5, min_minority=2)
ind_df_fast.head(10)

Pairs: 100%|██████████| 53/53 [17:03<00:00, 19.32s/it]

Usable pairs: 49
Mean BA over pairs: XGB = 0.751530612244898  | LSTM = 0.518061224489796
Median Δ (XGB-LSTM): 0.2466666666666668
% pairs XGB>LSTM: 0.7755102040816326


,person_id,exercise_id,n_total,n0,n1,BA_XGB_mean,BA_LSTM_mean,Delta_mean
0,9,1,22,11,11,0.966667,0.433333,0.533333
1,4,1,21,11,10,1.000000,0.500000,0.500000
2,2,4,21,10,11,0.966667,0.500000,0.466667
3,4,5,20,10,10,0.900000,0.433333,0.466667
4,5,5,25,12,13,0.891667,0.441667,0.450000
5,6,5,20,10,10,0.933333,0.500000,0.433333
6,3,6,20,10,10,0.966667,0.533333,0.433333
7,1,1,24,10,14,0.950000,0.541667,0.408333
8,5,2,20,10,10,0.900000,0.500000,0.400000
9,5,4,21,10,11,0.900000,0.500000,0.400000


In [29]:
print(ind_df_fast["BA_XGB_mean"].std())
print(ind_df_fast["BA_LSTM_mean"].std())

0.16178719616920637
0.08517027263157055


# SVM and RF

In [24]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold, StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [25]:
def fit_rf(X_train, y_train, seed=42):
    Xtr = X_train.reshape(len(X_train), -1)
    clf = RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        min_samples_leaf=1,
        class_weight="balanced",
        n_jobs=-1,
        random_state=int(seed),
    )
    clf.fit(Xtr, y_train)
    return clf

def predict_rf(model, X_test):
    Xte = X_test.reshape(len(X_test), -1)
    return model.predict(Xte)


def fit_svm(X_train, y_train, seed=42):
    # SVM needs scaling; use a pipeline
    Xtr = X_train.reshape(len(X_train), -1)
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(
            kernel="rbf",
            C=2.0,
            gamma="scale",
            class_weight="balanced",
            probability=False,
            random_state=int(seed)  # harmless; SVC is mostly deterministic unless prob=True
        ))
    ])
    clf.fit(Xtr, y_train)
    return clf

def predict_svm(model, X_test):
    Xte = X_test.reshape(len(X_test), -1)
    return model.predict(Xte)

In [55]:
def run_generalized_rf_svm(X, y, groups, seed=42, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)
    rows = []
    fold = 0

    for tr_idx, te_idx in gkf.split(X, y, groups):
        fold += 1
        Xtr, ytr = X[tr_idx], y[tr_idx]
        Xte, yte = X[te_idx], y[te_idx]

        s = int(seed + fold)

        rf = fit_rf(Xtr, ytr, seed=s)
        yhat_rf = predict_rf(rf, Xte)
        ba_rf = balanced_accuracy_score(yte, yhat_rf)

        svm = fit_svm(Xtr, ytr, seed=s)
        yhat_svm = predict_svm(svm, Xte)
        ba_svm = balanced_accuracy_score(yte, yhat_svm)

        rows.append({
            "fold": fold,
            "n_test": len(te_idx),
            "test_subjects": len(np.unique(groups[te_idx])),
            "BA_RF": ba_rf,
            "BA_SVM": ba_svm,
            "Delta_RF_minus_SVM": ba_rf - ba_svm
        })

    df = pd.DataFrame(rows)
    print(df)

    print("\nMean±std:")
    print("RF  :", df["BA_RF"].mean(), df["BA_RF"].std())
    print("SVM :", df["BA_SVM"].mean(), df["BA_SVM"].std())
    print("Δ   :", df["Delta_RF_minus_SVM"].mean(), df["Delta_RF_minus_SVM"].std())
    return df

gen_df_rf_svm = run_generalized_rf_svm(X, y, groups, seed=42, n_splits=5)

   fold  n_test  test_subjects     BA_RF    BA_SVM  Delta_RF_minus_SVM
0     1     135              2  0.533475  0.652150           -0.118674
1     2     229              2  0.591291  0.619255           -0.027964
2     3     227              2  0.534383  0.646538           -0.112155
3     4     233              2  0.479265  0.577627           -0.098362
4     5     233              2  0.519919  0.557502           -0.037583

Mean±std:
RF  : 0.5316667320475381 0.040155051437003826
SVM : 0.6106143916365342 0.04183144423391883
Δ   : -0.07894765958899612 0.04291918500945134


In [26]:
from tqdm import tqdm

def run_individualized_rf_svm(X, y, meta, seed=42, n_repeats=10, min_minority=2):
    meta2 = meta.copy()
    meta2["person_id"] = meta2["person_id"].astype(str)
    meta2["exercise_id"] = meta2["exercise_id"].astype(int)

    pair_groups = meta2.groupby(["person_id", "exercise_id"]).indices

    results = []
    for (pid, exid), idxs in tqdm(pair_groups.items(), total=len(pair_groups), desc="Pairs"):
        idxs = np.array(list(idxs))
        yy = y[idxs]
        n = len(idxs)

        n0 = int((yy == 0).sum())
        n1 = int((yy == 1).sum())
        minority = min(n0, n1)
        if minority < min_minority:
            continue

        # dynamic test size: at least 2 samples in test
        test_size = max(2, int(0.3 * n))
        if test_size >= n:
            continue

        sss = StratifiedShuffleSplit(
            n_splits=int(n_repeats),
            test_size=int(test_size),
            random_state=int(seed + (hash(pid) % 10000) + int(exid))
        )

        bas_rf, bas_svm = [], []
        for r, (tr_local, te_local) in enumerate(sss.split(np.zeros(n), yy)):
            tr_idx = idxs[tr_local]
            te_idx = idxs[te_local]

            Xtr, ytr = X[tr_idx], y[tr_idx]
            Xte, yte = X[te_idx], y[te_idx]

            s = int(seed + r + int(exid))

            rf = fit_rf(Xtr, ytr, seed=s)
            yhat_rf = predict_rf(rf, Xte)
            bas_rf.append(balanced_accuracy_score(yte, yhat_rf))

            svm = fit_svm(Xtr, ytr, seed=s)
            yhat_svm = predict_svm(svm, Xte)
            bas_svm.append(balanced_accuracy_score(yte, yhat_svm))

        results.append({
            "person_id": pid,
            "exercise_id": int(exid),
            "n_total": n,
            "n0": n0,
            "n1": n1,
            "BA_RF_mean": float(np.mean(bas_rf)),
            "BA_SVM_mean": float(np.mean(bas_svm)),
            "Delta_RF_minus_SVM": float(np.mean(bas_rf) - np.mean(bas_svm)),
            "RF_wins_frac": float(np.mean(np.array(bas_rf) > np.array(bas_svm))),
        })

    df = pd.DataFrame(results).sort_values("Delta_RF_minus_SVM", ascending=False).reset_index(drop=True)

    print("Usable pairs:", len(df))
    print("Mean BA over pairs: RF =", df["BA_RF_mean"].mean(), " | SVM =", df["BA_SVM_mean"].mean())
    print("Median Δ (RF-SVM):", df["Delta_RF_minus_SVM"].median())
    print("% pairs RF>SVM:", (df["Delta_RF_minus_SVM"] > 0).mean())

    return df

ind_df_rf_svm = run_individualized_rf_svm(X, y, meta, seed=42, n_repeats=10, min_minority=2)
ind_df_rf_svm.head(10)

Pairs: 100%|██████████| 53/53 [07:30<00:00,  8.49s/it]

Usable pairs: 49
Mean BA over pairs: RF = 0.9043197278911564  | SVM = 0.8992006802721088
Median Δ (RF-SVM): 0.0
% pairs RF>SVM: 0.4489795918367347


,person_id,exercise_id,n_total,n0,n1,BA_RF_mean,BA_SVM_mean,Delta_RF_minus_SVM,RF_wins_frac
0,2,3,8,3,5,1.000000,0.800000,0.200000,0.4
1,8,1,20,10,10,0.883333,0.750000,0.133333,0.7
2,5,2,20,10,10,0.950000,0.816667,0.133333,0.6
3,4,3,10,5,5,1.000000,0.875000,0.125000,0.4
4,9,6,23,5,18,0.920000,0.800000,0.120000,0.4
5,3,3,13,8,5,0.975000,0.875000,0.100000,0.2
6,2,1,20,10,10,1.000000,0.900000,0.100000,0.6
7,7,1,20,10,10,0.950000,0.850000,0.100000,0.5
8,8,3,10,5,5,1.000000,0.925000,0.075000,0.3
9,6,1,10,5,5,0.950000,0.875000,0.075000,0.3


In [28]:
print(ind_df_rf_svm["BA_RF_mean"].std())
print(ind_df_rf_svm["BA_SVM_mean"].std())

0.09841652458319956
0.09412870412016945


In [39]:
def summarize_two_models_generalized(df_gen):
    return pd.DataFrame([{
        "Regime": "Generalized (subject-disjoint)",
        "RF (BA mean±std)": f"{df_gen['BA_RF'].mean():.3f}±{df_gen['BA_RF'].std():.3f}",
        "SVM (BA mean±std)": f"{df_gen['BA_SVM'].mean():.3f}±{df_gen['BA_SVM'].std():.3f}",
        "Δ (RF-SVM) mean±std": f"{df_gen['Delta_RF_minus_SVM'].mean():.3f}±{df_gen['Delta_RF_minus_SVM'].std():.3f}",
    }])

def summarize_two_models_individualized(df_ind):
    return pd.DataFrame([{
        "Regime": "Individualized (per subject–exercise)",
        "RF (BA mean)": f"{df_ind['BA_RF_mean'].mean():.3f}",
        "SVM (BA mean)": f"{df_ind['BA_SVM_mean'].mean():.3f}",
        "Median Δ (RF-SVM)": f"{df_ind['Delta_RF_minus_SVM'].median():.3f}",
        "% pairs RF>SVM": f"{100*(df_ind['Delta_RF_minus_SVM']>0).mean():.1f}%",
    }])

summary_table = pd.concat([
    summarize_two_models_generalized(gen_df_rf_svm),
    summarize_two_models_individualized(ind_df_rf_svm)
], ignore_index=True)

summary_table

,Regime,RF (BA mean±std),SVM (BA mean±std),Δ (RF-SVM) mean±std,RF (BA mean),SVM (BA mean),Median Δ (RF-SVM),% pairs RF>SVM
0,Generalized (subject-disjoint),0.532±0.040,0.611±0.042,-0.079±0.043,NaN,NaN,NaN,NaN
1,Individualized (per subject–exercise),NaN,NaN,NaN,0.898,0.902,0.000,36.7%


# Temporal Shuffle

In [31]:
import numpy as np

def shuffle_time_within_sample(X, seed=42):
    """
    X: (N,T,A). For each sample n, permute time axis independently.
    """
    rng = np.random.default_rng(int(seed))
    Xs = X.copy()
    N, T, A = Xs.shape
    for n in range(N):
        perm = rng.permutation(T)
        Xs[n] = Xs[n, perm, :]
    return Xs

## RF and SVM

In [33]:
# Original
ind_rf_svm_orig = run_individualized_rf_svm(X, y, meta, seed=42, n_repeats=10, min_minority=2)

# Time-shuffled
X_shuf = shuffle_time_within_sample(X, seed=123)
ind_rf_svm_shuf = run_individualized_rf_svm(X_shuf, y, meta, seed=42, n_repeats=10, min_minority=2)

print("\n=== Individualized RF/SVM Stress Test (Time Shuffle) ===")
print("ORIG:  RF mean =", ind_rf_svm_orig["BA_RF_mean"].mean(), " RF STD ", ind_rf_svm_orig["BA_RF_mean"].std(), " | SVM mean =", ind_rf_svm_orig["BA_SVM_mean"].mean(),  " | SVM STD =", ind_rf_svm_orig["BA_SVM_mean"].std())
print("SHUF:  RF mean =", ind_rf_svm_shuf["BA_RF_mean"].mean(), " RF STD ", ind_rf_svm_shuf["BA_RF_mean"].std(), " | SVM mean =", ind_rf_svm_shuf["BA_SVM_mean"].mean(),  " | SVM STD =", ind_rf_svm_shuf["BA_SVM_mean"].std())
print("ΔRF (shuf-orig) =", ind_rf_svm_shuf["BA_RF_mean"].mean() - ind_rf_svm_orig["BA_RF_mean"].mean())
print("ΔSVM (shuf-orig) =", ind_rf_svm_shuf["BA_SVM_mean"].mean() - ind_rf_svm_orig["BA_SVM_mean"].mean())

Pairs: 100%|██████████| 53/53 [07:26<00:00,  8.43s/it]


Usable pairs: 49
Mean BA over pairs: RF = 0.9043197278911564  | SVM = 0.8992006802721088
Median Δ (RF-SVM): 0.0
% pairs RF>SVM: 0.4489795918367347


Pairs: 100%|██████████| 53/53 [07:26<00:00,  8.43s/it]

Usable pairs: 49
Mean BA over pairs: RF = 0.7819897959183673  | SVM = 0.8102721088435374
Median Δ (RF-SVM): 0.0
% pairs RF>SVM: 0.32653061224489793

=== Individualized RF/SVM Stress Test (Time Shuffle) ===
ORIG:  RF mean = 0.9043197278911564  RF STD  0.09841652458319956  | SVM mean = 0.8992006802721088  | SVM STD = 0.09412870412016945
SHUF:  RF mean = 0.7819897959183673  RF STD  0.1546027818259979  | SVM mean = 0.8102721088435374  | SVM STD = 0.15571997025568798
ΔRF (shuf-orig) = -0.12232993197278919
ΔSVM (shuf-orig) = -0.08892857142857136


In [56]:
gen_df_rf_svm_shuf = run_generalized_rf_svm(X_shuf, y, groups, seed=42, n_splits=5)

   fold  n_test  test_subjects     BA_RF    BA_SVM  Delta_RF_minus_SVM
0     1     135              2  0.546350  0.602441           -0.056090
1     2     229              2  0.564535  0.612907           -0.048372
2     3     227              2  0.503080  0.582255           -0.079175
3     4     233              2  0.508191  0.532504           -0.024314
4     5     233              2  0.462047  0.542905           -0.080857

Mean±std:
RF  : 0.5168405665936862 0.04003766820139339
SVM : 0.5746023072604693 0.035629188395451455
Δ   : -0.05776174066678326 0.023460965854756776


In [57]:
print("\n=== Generalized RF and SVM Stress Test (Time Shuffle) ===")
print("RF SHUF mean:", gen_df_rf_svm_shuf["BA_RF"].mean())
print("RF SHUF std:", gen_df_rf_svm_shuf["BA_RF"].std())

print("SVM SHUF mean:", gen_df_rf_svm_shuf["BA_SVM"].mean())
print("SVM SHUF std:", gen_df_rf_svm_shuf["BA_SVM"].std())

print("RF Δ (shuf-orig):", gen_df_rf_svm_shuf["BA_RF"].mean() - gen_df_rf_svm["BA_RF"].mean())
print("SVM Δ (shuf-orig):", gen_df_rf_svm_shuf["BA_SVM"].mean() - gen_df_rf_svm["BA_SVM"].mean())


=== Generalized RF and SVM Stress Test (Time Shuffle) ===
RF SHUF mean: 0.5168405665936862
RF SHUF std: 0.04003766820139339
SVM SHUF mean: 0.5746023072604693
SVM SHUF std: 0.035629188395451455
RF Δ (shuf-orig): -0.014826165453851958
SVM Δ (shuf-orig): -0.036012084376064846


## LSTM

In [34]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm

def run_individualized_lstm_fast(X, y, meta, seed=42, n_repeats=5, min_minority=2):
    meta2 = meta.copy()
    meta2["person_id"] = meta2["person_id"].astype(str)
    meta2["exercise_id"] = meta2["exercise_id"].astype(int)

    pair_groups = meta2.groupby(["person_id", "exercise_id"]).indices

    rows = []
    for (pid, exid), idxs in tqdm(pair_groups.items(), total=len(pair_groups), desc="Pairs (LSTM)"):
        idxs = np.array(list(idxs))
        yy = y[idxs]
        n = len(idxs)

        n0 = int((yy==0).sum()); n1 = int((yy==1).sum())
        if min(n0, n1) < min_minority:
            continue

        test_size = max(2, int(0.3 * n))
        if test_size >= n:
            continue

        sss = StratifiedShuffleSplit(
            n_splits=int(n_repeats),
            test_size=int(test_size),
            random_state=int(seed + (hash(pid) % 10000) + int(exid))
        )

        bas = []
        for r, (tr_local, te_local) in enumerate(sss.split(np.zeros(n), yy)):
            tr_idx = idxs[tr_local]
            te_idx = idxs[te_local]

            Xtr, ytr = X[tr_idx], y[tr_idx]
            Xte, yte = X[te_idx], y[te_idx]

            lstm = fit_lstm_fast(Xtr, ytr, seed=int(seed + r + int(exid)))
            yhat = predict_lstm(lstm, Xte)
            bas.append(balanced_accuracy_score(yte, yhat))

        rows.append({
            "person_id": pid,
            "exercise_id": int(exid),
            "BA_LSTM_mean": float(np.mean(bas))
        })

    df = pd.DataFrame(rows)
    print("Usable pairs:", len(df))
    print("Mean BA over pairs (LSTM):", df["BA_LSTM_mean"].mean())
    return df

In [35]:
# Original
ind_lstm_orig = run_individualized_lstm_fast(X, y, meta, seed=42, n_repeats=5, min_minority=2)

# Time-shuffled
ind_lstm_shuf = run_individualized_lstm_fast(X_shuf, y, meta, seed=42, n_repeats=5, min_minority=2)

print("\n=== Individualized LSTM Stress Test (Time Shuffle) ===")
print("ORIG mean:", ind_lstm_orig["BA_LSTM_mean"].mean())
print("SHUF mean:", ind_lstm_shuf["BA_LSTM_mean"].mean())
print("ORIG std:", ind_lstm_orig["BA_LSTM_mean"].std())
print("SHUF std:", ind_lstm_shuf["BA_LSTM_mean"].std())
print("Δ (shuf-orig):", ind_lstm_shuf["BA_LSTM_mean"].mean() - ind_lstm_orig["BA_LSTM_mean"].mean())

Pairs (LSTM): 100%|██████████| 53/53 [22:01<00:00, 24.93s/it]


Usable pairs: 49
Mean BA over pairs (LSTM): 0.5180612244897959


Pairs (LSTM): 100%|██████████| 53/53 [27:54<00:00, 31.59s/it]

Usable pairs: 49
Mean BA over pairs (LSTM): 0.5217006802721088

=== Individualized LSTM Stress Test (Time Shuffle) ===
ORIG mean: 0.5180612244897959
SHUF mean: 0.5217006802721088
ORIG std: 0.08517027263157055
SHUF std: 0.09687028948877306
Δ (shuf-orig): 0.0036394557823129503


In [46]:
gen_df_shuf = run_generalized(X_shuf, y, groups, seed=42, n_splits=5)

   fold  n_test  test_subjects    BA_XGB   BA_LSTM  Delta_XGB_minus_LSTM
0     1     135              2  0.565719  0.534819              0.030900
1     2     229              2  0.549112  0.577271             -0.028159
2     3     227              2  0.557968  0.592235             -0.034266
3     4     233              2  0.532652  0.503210              0.029442
4     5     233              2  0.529268  0.558758             -0.029490

Mean±std:
XGB : 0.5469438529185846 0.015775110799296766
LSTM: 0.5532583858694246 0.03525214468230461
Δ   : -0.006314532950839969 0.033388026050044234


In [47]:
gen_df_shuf

,fold,n_test,test_subjects,BA_XGB,BA_LSTM,Delta_XGB_minus_LSTM
0,1,135,2,0.565719,0.534819,0.030900
1,2,229,2,0.549112,0.577271,-0.028159
2,3,227,2,0.557968,0.592235,-0.034266
3,4,233,2,0.532652,0.503210,0.029442
4,5,233,2,0.529268,0.558758,-0.029490


In [59]:
print("\n=== Generalized XGB and LSTM Stress Test (Time Shuffle) ===")
print("LSTM SHUF mean:", gen_df_shuf["BA_LSTM"].mean())
print("LSTM SHUF std:", gen_df_shuf["BA_LSTM"].std())

print("XGB SHUF mean:", gen_df_shuf["BA_XGB"].mean())
print("XGB SHUF std:", gen_df_shuf["BA_XGB"].std())

print("LSTM Δ (shuf-orig):", gen_df_shuf["BA_LSTM"].mean() - gen_df["BA_LSTM"].mean())
print("XGB Δ (shuf-orig):", gen_df_shuf["BA_XGB"].mean() - gen_df["BA_XGB"].mean())


=== Generalized XGB and LSTM Stress Test (Time Shuffle) ===
LSTM SHUF mean: 0.5532583858694246
LSTM SHUF std: 0.03525214468230461
XGB SHUF mean: 0.5469438529185846
XGB SHUF std: 0.015775110799296766
LSTM Δ (shuf-orig): 0.04925146333672503
XGB Δ (shuf-orig): 0.01643887861952731


## XGB

In [61]:
def run_individualized_xgb_fast(X, y, meta, seed=42, n_repeats=10, min_minority=2):
    meta2 = meta.copy()
    meta2["person_id"] = meta2["person_id"].astype(str)
    meta2["exercise_id"] = meta2["exercise_id"].astype(int)

    pair_groups = meta2.groupby(["person_id", "exercise_id"]).indices
    results = []

    for (pid, exid), idxs in tqdm(pair_groups.items(), total=len(pair_groups), desc="Pairs (XGB)"):
        idxs = np.array(list(idxs))
        yy = y[idxs]
        n = len(idxs)

        n0 = int((yy==0).sum())
        n1 = int((yy==1).sum())
        if min(n0, n1) < min_minority:
            continue

        test_size = max(2, int(0.3 * n))
        if test_size >= n:
            continue

        sss = StratifiedShuffleSplit(
            n_splits=int(n_repeats),
            test_size=int(test_size),
            random_state=int(seed + (hash(pid) % 10000) + int(exid))
        )

        bas = []

        for r, (tr_local, te_local) in enumerate(sss.split(np.zeros(n), yy)):
            tr_idx = idxs[tr_local]
            te_idx = idxs[te_local]

            Xtr, ytr = X[tr_idx], y[tr_idx]
            Xte, yte = X[te_idx], y[te_idx]

            model = fit_xgb_fast(Xtr, ytr, seed=int(seed + r + int(exid)))
            yhat = predict_xgb(model, Xte)

            bas.append(balanced_accuracy_score(yte, yhat))

        results.append({
            "person_id": pid,
            "exercise_id": exid,
            "BA_XGB_mean": float(np.mean(bas))
        })

    df = pd.DataFrame(results)
    print("Usable pairs:", len(df))
    print("Mean BA over pairs (XGB):", df["BA_XGB_mean"].mean())
    return df

In [63]:
# Original
ind_xgb_orig = run_individualized_xgb_fast(X, y, meta, seed=42, n_repeats=10)

# Shuffled
ind_xgb_shuf = run_individualized_xgb_fast(X_shuf, y, meta, seed=42, n_repeats=10)

print("\n=== Individualized XGB Stress Test ===")
print("ORIG mean:", ind_xgb_orig["BA_XGB_mean"].mean())
print("SHUF mean:", ind_xgb_shuf["BA_XGB_mean"].mean())
print("ORIG std:", ind_xgb_orig["BA_XGB_mean"].std())
print("SHUF std:", ind_xgb_shuf["BA_XGB_mean"].std())
print("XGB Δ (shuf-orig):", ind_xgb_shuf["BA_XGB_mean"].mean() - ind_xgb_orig["BA_XGB_mean"].mean())

Pairs (XGB): 100%|██████████| 53/53 [00:50<00:00,  1.04it/s]


Usable pairs: 49
Mean BA over pairs (XGB): 0.74906462585034


Pairs (XGB): 100%|██████████| 53/53 [00:42<00:00,  1.24it/s]

Usable pairs: 49
Mean BA over pairs (XGB): 0.6420068027210885

=== Individualized XGB Stress Test ===
ORIG mean: 0.74906462585034
SHUF mean: 0.6420068027210885
ORIG std: 0.14238752796768506
SHUF std: 0.12250400578854705
XGB Δ (shuf-orig): -0.1070578231292515
